In [4]:
import numpy as np
import pandas as pd


from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

In [8]:
df = pd.read_csv("Crop_recommendation.csv")
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [9]:
X = df.drop(["label"], axis=1)
y = pd.DataFrame(df.label)

oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
oh_cols = pd.DataFrame(oh_encoder.fit_transform(y))

# Return the index
oh_cols.index = y.index

y = oh_cols

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y_train)

predictions = model.predict(X_val)

accuracy = accuracy_score(y_val, predictions)

print("accuracy: ", accuracy)

KeyError: "['label'] not found in axis"

In [7]:
original_df = df.copy()

oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_train

NameError: name 'df' is not defined